In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [5]:
# Create a Sequential model
model = Sequential()

# Add the first convolutional block
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

# Add the second convolutional block
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))

# # Add the third convolutional block
# model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(MaxPooling2D((2, 2)))

# Flatten the feature maps
model.add(Flatten())

# Add a fully connected (dense) layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization

# Add the output layer with 10 units (assuming 10 classes for classification)
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 conv2d_5 (Conv2D)           (None, 224, 224, 32)      9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 112, 112, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 conv2d_7 (Conv2D)           (None, 112, 112, 64)      36928     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                 

In [6]:
# Data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load your dataset using the ImageDataGenerator
train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Edge projects/Leaf Disease/Tomato_Train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Edge projects/Leaf Disease/Tomato_test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 8950 images belonging to 10 classes.
Found 2295 images belonging to 10 classes.


In [ ]:
# # Train the model
# history = model.fit(
#     train_generator,
#     steps_per_epoch=len(train_generator),
#     epochs=50,
#     validation_data=validation_generator,
#     validation_steps=len(validation_generator),
#     #callbacks=[reduce_lr]
#     # validation_split = 0.3,
#     # validation_steps=(len(train_generator)/32)*0.3
# )

from tensorflow.keras.callbacks import ModelCheckpoint

# Define a callback to save the best model
checkpoint = ModelCheckpoint("/content/drive/MyDrive/Edge projects/Leaf Disease/Models/2_best_minet_model_2.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# Train the model with the callback
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[checkpoint]
)


Epoch 1/50
280/280 [==============================] - ETA: 0s - loss: 1.7914 - accuracy: 0.3692 
Epoch 1: val_loss improved from inf to 1.23898, saving model to /content/drive/MyDrive/Edge projects/Leaf Disease/Models/2_best_minet_model_2.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


280/280 [==============================] - 3952s 14s/step - loss: 1.7914 - accuracy: 0.3692 - val_loss: 1.2390 - val_accuracy: 0.5682
Epoch 2/50
280/280 [==============================] - ETA: 0s - loss: 1.1455 - accuracy: 0.6028
Epoch 2: val_loss improved from 1.23898 to 0.84253, saving model to /content/drive/MyDrive/Edge projects/Leaf Disease/Models/2_best_minet_model_2.h5
280/280 [==============================] - 211s 754ms/step - loss: 1.1455 - accuracy: 0.6028 - val_loss: 0.8425 - val_accuracy: 0.7081
Epoch 3/50
280/280 [==============================] - ETA: 0s - loss: 0.8905 - accuracy: 0.6829
Epoch 3: val_loss improved from 0.84253 to 0.68910, saving model to /content/drive/MyDrive/Edge projects/Leaf Disease/Models/2_best_minet_model_2.h5
280/280 [==============================] - 212s 759ms/step - loss: 0.8905 - accuracy: 0.6829 - val_loss: 0.6891 - val_accuracy: 0.7534
Epoch 4/50
280/280 [==============================] - ETA: 0s - loss: 0.7603 - accuracy: 0.7289
Epoch 4: v

In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/Edge projects/Leaf Disease/Models/2.h5')